# Cloud classification 

Competition link: https://www.kaggle.com/c/understanding_cloud_organization

In this notebook, I attempt to train a unet model with real-time image augmentation.

In [32]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator

import skimage.io as io
import skimage.transform as trans
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

Using TensorFlow backend.


In [5]:
data_path = './data'
train_csv_path = os.path.join(data_path,'train.csv')
train_image_path = os.path.join(data_path,'train_images')

In [7]:
train = pd.read_csv('./data/train.csv')
train = train.fillna(-1)
train[['Image', 'Label']] = train['Image_Label'].str.split('_', expand=True)
train['Image'] = train['Image'].str.split('.').str[0]
train = train.drop('Image_Label', axis=1)
train.head()

,EncodedPixels,Image,Label
0,264918 937 266318 937 267718 937 269118 937 27...,0011165,Fish
1,1355565 1002 1356965 1002 1358365 1002 1359765...,0011165,Flower
2,-1,0011165,Gravel
3,-1,0011165,Sugar
4,233813 878 235213 878 236613 878 238010 881 23...,002be4f,Fish


In [47]:
def rle2mask(row, height=1400, width=2100):
    '''
    convert RLE(run length encoding) string to numpy array

    Parameters: 
    rle_string (str): string of rle encoded mask
    height (int): height of the mask
    width (int): width of the mask 

    Returns: 
    numpy.array: numpy array of the mask
    '''
    if row['EncodedPixels'] == -1:
        return
    rows, cols = height, width
    rle_string = row['EncodedPixels']
    
    rle_numbers = [int(num_string) for num_string in rle_string.split(' ')]
    rle_pairs = np.array(rle_numbers).reshape(-1,2)
    img = np.zeros(rows*cols, dtype=np.uint8)
    for index, length in rle_pairs:
        index -= 1
        img[index:index+length] = 255
    img = img.reshape(cols,rows)
    img = img.T
    
    pil_image = Image.fromarray(img)
    
    label = row['Label']
    mask_file = row['Label'] + '_' + row['Image'] + '.jpg'
    
    if label == 'Fish':
        pil_image.save('./data/mask_fish/' + mask_file)
    elif label == 'Flower':
        pil_image.save('./data/mask_flower/' + mask_file)
    elif label == 'Gravel':
        pil_image.save('./data/mask_gravel/' + mask_file)
    elif label == 'Sugar':
        pil_image.save('./data/mask_sugar/' + mask_file)
    
    if row.name % 500 == 0:
        print('On row:', row.name)

In [48]:
train.apply(rle2mask, axis=1)

0
2500
3500
5000
6000
6500
9000
9500
10500
11000
11500
12000
12500
14000
14500
15000
15500
17000
19000
20500
22000


0        None
1        None
2        None
3        None
4        None
5        None
6        None
7        None
8        None
9        None
10       None
11       None
12       None
13       None
14       None
15       None
16       None
17       None
18       None
19       None
20       None
21       None
22       None
23       None
24       None
25       None
26       None
27       None
28       None
29       None
         ... 
22154    None
22155    None
22156    None
22157    None
22158    None
22159    None
22160    None
22161    None
22162    None
22163    None
22164    None
22165    None
22166    None
22167    None
22168    None
22169    None
22170    None
22171    None
22172    None
22173    None
22174    None
22175    None
22176    None
22177    None
22178    None
22179    None
22180    None
22181    None
22182    None
22183    None
Length: 22184, dtype: object

## Creating ImageDataGenerator objects

Reference: https://towardsdatascience.com/a-keras-pipeline-for-image-segmentation-part-1-6515a421157d
https://keras.io/preprocessing/image/

Keras has a image preprocessing class called `ImageDataGenerator` that when passed into the model, augments the images during model fitting. For now, we will rescale, shear, zoom, rotate, or flip the images.

A validation set still needs to be created.

In [87]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=90,
        horizontal_flip=True,
        vertical_flip=True)

mask_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=90,
        horizontal_flip=True,
        vertical_flip=True)

train_image_generator = train_datagen.flow_from_directory(
                        './data/train_images',
                        class_mode=None,
                        color_mode='grayscale',
                        batch_size = 4)

train_mask_generator = train_datagen.flow_from_directory(
                        './data/train_masks',
                        class_mode=None,
                        color_mode='grayscale',
                        batch_size = 4)

train_generator = zip(train_image_generator, train_mask_generator)

Found 5546 images belonging to 1 classes.
Found 11836 images belonging to 4 classes.


Unet obtained from https://github.com/zhixuhao/unet/blob/master/model.py

In [88]:
def unet(pretrained_weights = None, input_size = (256,256,1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(input = inputs, outputs = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    #model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model

In [89]:
model = unet()

/Users/sampsonmao/miniconda3/envs/dss/lib/python3.7/site-packages/ipykernel_launcher.py:43: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`


In [91]:
model.fit_generator(
        train_generator,
        steps_per_epoch=20,
        epochs=1)

Epoch 1/1
20/20 [==============================] - 265s 13s/step - loss: 0.6915 - accuracy: 0.7102


In [97]:
model.get_weights()

[array([[[[ 1.02662265e+00,  8.48559976e-01,  3.18417698e-01,
            3.27307582e-01, -3.46181020e-02,  9.08523560e-01,
            5.28360665e-01, -2.92566389e-01,  6.95352435e-01,
            6.18281424e-01,  2.54811738e-02,  7.84320652e-01,
           -4.81138879e-04,  7.38437116e-01,  1.03258765e+00,
            1.81515455e-01,  6.65600538e-01, -4.56829906e-01,
            2.04660758e-01, -2.02704132e-01, -2.20504209e-01,
            1.80762827e-01, -4.49270308e-01,  1.02409065e+00,
            9.86937955e-02,  1.26322240e-01,  3.72788915e-03,
           -5.66803038e-01,  4.34013069e-01,  6.71470389e-02,
           -3.75277787e-01,  4.99374777e-01,  7.28162304e-02,
           -4.87250179e-01, -9.47296858e-01, -2.18408108e-01,
           -1.56473033e-02,  8.70595574e-01,  1.05832148e+00,
           -5.11708319e-01,  4.08506572e-01,  7.83147290e-02,
           -1.00044958e-01, -3.06875199e-01,  7.17455864e-01,
            1.83743268e-01,  2.27098688e-01,  2.95020938e-01,
        

In [96]:
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_204 (Conv2D)             (None, 256, 256, 64) 640         input_10[0][0]                   
__________________________________________________________________________________________________
conv2d_205 (Conv2D)             (None, 256, 256, 64) 36928       conv2d_204[0][0]                 
__________________________________________________________________________________________________
max_pooling2d_37 (MaxPooling2D) (None, 128, 128, 64) 0           conv2d_205[0][0]                 
____________________________________________________________________________________________